## Assignment 5

Assigned: 13 November 2018  
Due: 4 December 2018

In this assignment you will design a database and write functions to query it. I have provided data for 999 books from Tar Heel Reader. The books are provided in JSON format which is easy to read in Python. 

In [346]:
# setup
import sqlite3
import comp521

check, report = comp521.start('A5')

import json
books = json.load(open('books.json'))

Now the variable `books` is an array of dictionaries. A typical book looks like this:

     {u'categories': [u'Animals and Nature'],
     u'created': u'2015-04-02 07:57:35',
     u'email': u'cwhitei0@miitbeian.gov.cn',
     u'first_name': u'Christopher',
     u'language': u'es',
     u'last_name': u'White',
     u'login': u'cwhitei0',
     u'pages': [{u'height': 310,
       u'caption': u'La rosa m\xe1s hermosa',
       u'url': u'/cache/images/48/3329876048_2b480e2243.jpg',
       u'width': 500},
      {u'height': 375,
       u'caption': u'Aunque crezca en un desierto',
       u'url': u'/cache/images/19/5886437819_01932b8f38.jpg',
       u'width': 500},
      {u'height': 500,
       u'caption': u'siempre tiene donde estar',
       u'url': u'/cache/images/17/5886448317_fcbf6c7d4d.jpg',
       u'width': 375}],
     u'title': u'La rosa'}
 
`categories` is a list of strings each indicating a library-like classification for the book. The list may be empty. Assume that additional categories may be created in the future.

`email`, `first_name`, `last_name`, and `login` relate to the author of the book. Of course, one author may write many books and the information must remain consistent. `login` is unique for every author.

`language` is the two-letter code for the language of the book. 

`title` is, of course, the title of the book and certainly not unique.

`pages` is an array. For each page of the book we have several fields. `caption` is the caption or text for this page of the book. `url` is the URL for the picture and its dimensions are in `width` and `height`. The dimensions of a particular image are always the same. There is significant reuse of pictures across books. The `url` for a picture sometimes changes and must be changed across all instances.

## Your mission

Your mission is to design a database in BCNF, write code to load it from the `books` array, and code to query it in various ways as described below.

Imagine you are implemeting the database part of the backend for Tar Heel Reader.

In [347]:
# I'll create an in-memory database for you. Do all your work in here
import sqlite3
db = sqlite3.Connection(':memory:')
cursor = db.cursor()

## 1. Design the Database

Create a set of tables with schemas that are in Boyce-Codd Normal Form (_the values are determined by the key, the whole key, and nothing but the key_) or as close to it as you can reasonably get. Use the same names for attributes as in the JSON format data. Give any new attributes you create reasonably descriptive names.

I'm going to assume below that you can find a given book by a unique integer id that you (or the DB) will assign.

In [348]:
# Write code here to create the tables
# use the db variable from above
cursor.execute('DROP table IF EXISTS authors')
cursor.execute('DROP table IF EXISTS books')
cursor.execute('DROP table IF EXISTS pages')
cursor.execute('DROP table IF EXISTS categories')
cursor.execute('DROP table IF EXISTS images')


cursor.execute(
    """
    CREATE TABLE authors(
    login text primary key, 
    email text, 
    first_name text, 
    last_name text
    )
    """)

cursor.execute(
    """
    CREATE TABLE books(
    bid integer primary key, 
    created text, 
    language text, 
    title text, 
    login text, 
    numOfPages INTEGER,
    foreign key (login) references authors(login)
    )
    """)

cursor.execute(
    """
    CREATE TABLE categories(
    category text, 
    bid integer, 
    primary key (category, bid), 
    foreign key (bid) references authors(bid)
    )
    """)

cursor.execute(
    """
    CREATE TABLE pages(
    pid integer primary key, 
    height integer, 
    caption text, 
    url text, 
    width integer, 
    bid integer, 
    foreign key (bid) references books(bid), 
    foreign key (url) references images(url)
    )
    """)


cursor.execute(
    """
    CREATE TABLE images(
    url text primary key
    )
    """)


## 2. Load the data into your database. 

You should not need additional datastructures besides the db itself to accomplish this. 

In [349]:
# Write your code here, use the same db as above
# the source data is already in the variable books
# you'll, of course, need to iterate over it.

#had to use .format, other ways didnt work :/
for row in books:
    x = cursor.execute(
        """
        SELECT * 
        FROM authors 
        WHERE login ='{}'
        """.format(row['login']))
    
    if not x.fetchone():
        cursor.execute(
            """
            INSERT INTO authors (
            login, email, first_name, last_name
            ) 
            VALUES ('{}', '{}', '{}', '{}')
            """.format(row['login'], row['email'], row['first_name'], row['last_name']))
        
    cursor.execute(
        """
        INSERT 
        INTO books (
            created, language, title, login
        ) 
        VALUES ('{}', '{}', '{}', '{}')
        """.format(row['created'], row['language'], row['title'].replace("'", "''"), row['login']))
        
    lastbid = cursor.lastrowid
    for cat in row['categories']:
        cursor.execute(
            """
            INSERT 
            INTO categories 
            (category, bid) 
            VALUES ('{}', '{}')
            """.format(cat, lastbid))
            
        
    for page in row['pages']:
        url = cursor.execute(
            """
            SELECT * 
            FROM images 
            WHERE url = '{}'
            """.format(page['url']))
        
        if not url.fetchone():
            cursor.execute(
                """
                INSERT 
                INTO images (url) 
                VALUES ('{}')
                """.format(page['url']))
            
        cursor.execute(
            """
            INSERT 
            INTO pages
            (height, caption, url, width, bid, url) 
            VALUES ('{}', '{}', '{}', '{}', '{}', '{}')
            """.format(page['height'], page['caption'].replace("'", "''"), page['url'], page['width'], lastbid, page['url']))
        


In [350]:
# a helper for the questions below
def getTableSize(name):
    return cursor.execute('''select count(*) from %s''' % name).fetchone()[0]

# How many entries are in your authors table?
NumberOfAuthors = getTableSize('authors')
check("Part 2 Number Of Authors", NumberOfAuthors, points=5)

# How many images are in your table?
NumberOfImages = getTableSize('images')
check("Part 2 Number Of Images", NumberOfImages, points=5)

# How many pages?
NumberOfPages = getTableSize('pages')
check("Part 2 Number of Pages", NumberOfPages, points=5)

Part 2 Number Of Authors appears correct
Part 2 Number Of Images appears correct
Part 2 Number of Pages appears correct


## 3. List the categories with the number of books in each for a given language.

A visitor to Tar Heel Reader might want to see how many books in their native language are available in each category.

Fill in the body of the function so that it uses a single query to produce a list of tuples with the category first and the number of books in that category second. Sort by category in ascending order.

For my table design I was able to do this with a single SQL statement. 

Always use the ? parameter substitution form to avoid SQL injection attacks.

In [351]:
def listCategoryCounts(language):
    #Write your code here
    cursor.execute(
        """
        SELECT c.category, count(distinct b.bid)
        FROM categories c, books b
        WHERE c.bid = b.bid AND b.language=?
        GROUP by c.category
        ORDER by c.category ASC
        """,[language])
    
    return cursor.fetchall()

r = listCategoryCounts('en')

check('Part 3', r, points=20)

r

Part 3 appears correct


[('Alphabet', 64),
 ('Animals and Nature', 257),
 ('Art and Music', 41),
 ('Biographies', 28),
 ('Fairy and Folk Tales', 24),
 ('Fiction', 187),
 ('Foods', 58),
 ('Health', 22),
 ('History', 43),
 ('Holidays', 55),
 ('Math and Science', 80),
 ('Nursery Rhymes', 2),
 ('People and Places', 243),
 ('Poetry', 25),
 ('Recreation and Leisure', 111),
 ('Sports', 48)]

## 4. List the books in a category.

The next step for our visitor is to select a category and browse the available books. We don't want to send __all__ the books, that would take too long, so we will send only 12 books (a chunk) at a time.

Fill in the body of the function below so that it returns a list of tuples with the book's unique id, title, author's last name, page count, and the URL of the picture on the first page of the book. The frontend will use this information to construct a grid of book previews with a link to read each book.The books should be ordered with the most recently created books first.

You should do this with a single query.

Your result should look like this:

    [(u'Farm Animals', u'Murphy', 980, 8, u'/cache/images/89/291005289_a05ae73053.jpg'),
     ...
    ]


In [352]:
def listBooks(language, category, chunk):
    # language is the 2-letter language code
    # category is a category name
    # chunk is an integer. 0 means the first 12 books, 1 means the next 12, etc.
    # write your code here
    cursor.execute(
        """
        SELECT B.bid, B.title, A.last_name, counts.count, E.url
        FROM books B, authors A, Categories C, 
            (
            SELECT P.url AS url, G.bid AS bid 
            FROM 
                (
                SELECT B.title AS title, B.bid AS bid, MIN(P.pid) AS min, B.title AS title
                FROM pages P, books B
                WHERE P.bid = B.bid
                GROUP BY B.bid
                ) G, Pages P
                WHERE P.pid = G.min AND P.bid = G.bid
            ) E,
            (
            SELECT COUNT(p.pid) AS count, B.bid AS bid 
            FROM Books B, pages P 
            WHERE P.bid = B.bid 
            GROUP BY B.bid
            ) 
            counts
        WHERE C.bid = B.bid AND C.category = ? AND B.language = ? AND E.bid = B.bid AND B.login = A.login AND counts.bid = B.bid 
        ORDER BY B.created DESC
        """,[category, language])

    
    return cursor.fetchall()[(chunk*12):((chunk+1)*12)]

r1 = listBooks('en', 'Animals and Nature', 0)
# I'm stripping off the ID before testing
check('Part 4 page 1', [ r[1:] for r in r1 ], points=10)

r2 = listBooks('en', 'Animals and Nature', 1)
toRead = r2[-1][0] # get the id of the last book on the second page, we'll read it below
# I'm stripping off the ID before testing
check('Part 4 page 2', [ r[1:] for r in r2], points=10)

r1

Part 4 page 1 appears correct
Part 4 page 2 appears correct


[(999,
  'Seaside Aquarium',
  'Simpson',
  8,
  '/cache/images/21/122430121_a653d2f8ab.jpg'),
 (998,
  'The Funny Crab',
  'Bennett',
  5,
  '/cache/images/49/5638860949_d89877e19d.jpg'),
 (994,
  'Polar Bears',
  'Rogers',
  10,
  '/cache/images/70/3582475670_df8e8e8974.jpg'),
 (993, 'Horse', 'Rice', 6, '/cache/images/79/57632579_c7482974fe.jpg'),
 (990, 'The Mouse', 'Kelly', 13, '/cache/images/39/8093539_473996654f.jpg'),
 (936, 'Cats', 'Patterson', 8, '/cache/images/70/5110834170_0797f39278.jpg'),
 (987,
  'The Tired Lion',
  'Bennett',
  4,
  '/cache/images/57/5657669257_d4be2a3f98.jpg'),
 (977,
  'My book on how to help the Earth.',
  'Harper',
  8,
  '/cache/images/48/2435883148_0eba86944f.jpg'),
 (976,
  'Help the Earth',
  'Harper',
  7,
  '/cache/images/77/3717759677_4a520a1dbb.jpg'),
 (975,
  'My book about how to save the Earth',
  'Harper',
  8,
  '/cache/images/77/3717759677_4a520a1dbb.jpg'),
 (986,
  'Farm Animals',
  'Reynolds',
  8,
  '/cache/images/89/291005289_a05ae7

## 5. Fetch a single book

Now that our vistor has clicked on a book, we want to fetch all of its content so the Javascript in the browser can render it without further reference to the server. We want to produce a result formatted like the ones we read in earlier so create a Python dictionary and assign values to keys with the same names. Exclude the `id` from the result to make testing easier. Finally return a JSON encoded string, I've included that code for you inside the function. 

I can't see any way to do this step with only one query. I got it down to three. That isn't so bad because after this step our visitor will be happily reading a book without bothering us again for a while.

Fill in the body of the function below so that it produces a JSON encoded string that looks like one of the entries in your original books array. Do __not__ simply return an item from that array, you __must__ reconstruct the book from the data in your database.

In [353]:
def fetchBook(id):
    book = {} # fill in the values here
    
    #Write your code here
    
    #I had to do some weird stuff here...
    categories = db.cursor().execute(
        """
        SELECT * 
        FROM categories 
        WHERE bid = ?
        """,[id]).fetchall()    
    
    books = db.cursor().execute(
        """
        SELECT * 
        FROM authors, books 
        WHERE books.bid = ? AND authors.login = books.login
        """,[id]).fetchall()
    
    pages = db.cursor().execute(
        """
        SELECT * 
        FROM pages, books 
        WHERE books.bid = ? AND pages.bid = books.bid
        """,[id]).fetchall()

    
    #put into books
    book["categories"] = [c[0] for c in categories]
    book["login"] = books[0][0]
    book["email"] = books[0][1]
    book["first_name"] = books[0][2]
    book["last_name"] = books[0][3]
    book["created"] = books[0][5]
    book["language"] = books[0][6]
    book["title"] = books[0][7]
    book["pages"] = [
        {
            "caption" : p[2], 
             "height" : p[1], 
            "url" : p[3], 
            "width": p[4]
        }  
        for p in pages
    ]
    
    
    ###
    return json.dumps(book, sort_keys=True)
    
b = fetchBook(toRead)
check('Part 5', b, points=30)

Part 5 appears correct


## 6. List the books by an author.

The authors at our site like to see a list of the books they have written. Fill in the code below to return a list of the books in the same format as listBooks above in part 3.

In [354]:
def listAuthor(login):
    # login is the unique login for an author
    bids = db.cursor().execute(
        """
        SELECT books.bid 
        FROM authors, books 
        WHERE authors.login = ? AND authors.login = books.login
        """,[login]).fetchall()
    
    results = []
    for bid in bids:
        cursor = db.cursor().execute(
            """
            SELECT B.bid, B.title, A.last_name, counts.count, E.url
            FROM books B, authors A, Categories C, 
                (
                SELECT P.url AS url, G.bid AS bid 
                FROM 
                    (
                    SELECT B.title AS title, B.bid AS bid, MIN(P.pid) AS min, B.title AS title
                    FROM pages P, books B
                    WHERE P.bid = B.bid
                    GROUP BY B.bid
                    ) G, Pages P
                    WHERE P.pid = G.min AND P.bid = G.bid) E,
                (
                SELECT COUNT(p.pid) AS count, B.bid AS bid 
                FROM Books B, pages P 
                WHERE P.bid = B.bid GROUP BY B.bid
                ) counts
                WHERE E.bid = B.bid AND B.login = A.login AND C.bid = B.bid AND counts.bid = B.bid AND B.bid = ?
                ORDER BY B.created DESC
            """,[bid[0]]
        )
        
        results.append(cursor.fetchall()[0])
            
    x = sorted(results, key=lambda x: x[3], reverse=False)

    return x

r1 = listAuthor('jhansenrr')
# I'm stripping off the ID before testing
check('Part 6', [ r[1:] for r in r1 ], points=15)

r1

Part 6 appears correct


[(1,
  'PLURALS: cuniculus ursum amat.',
  'Hansen',
  8,
  '/cache/images/06/357393306_bb325a5c92.jpg'),
 (3,
  'Plurals: cuniculus et amicus',
  'Hansen',
  8,
  '/cache/images/67/219674667_b3aeb1528e.jpg'),
 (2,
  'Plurals: Canes, nomine Montius et Daisia',
  'Hansen',
  9,
  '/uploads/2009/07/annapmagistra-1246609542.jpg'),
 (15,
  'Cuniculus versipellem salutat',
  'Hansen',
  14,
  '/cache/images/39/3178513639_d71ae707ab.jpg')]

In [355]:
onyen = 'armaan'
collaborators = ['lexokan']
report(onyen, collaborators)

  Collaborators: ['lexokan']
Report for armaan
  8 of 8 appear correct, 100 of 100 points
